# **Creating Association Table**

## Original Token Association Tables

In [ ]:
#  [1]
import re
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

In [ ]:
# TokenassociationTables
#  [2] new: SIMnoun_500 as pandas
#V2.2 itterating through associations and built table 

target_corpus = "/compLing/students/hiwi-theses/data/aylin.wahl/MUDCATData/original_data/targets_Brysbaert_filtered/targets_Brysbaert_filtered_adj.txt"
#target_corpus = "/compLing/students/hiwi-theses/data/aylin.wahl/MUDCATData/SIMnoun_mini.txt"  #try-mini corpus
targetdata = pd.read_csv(target_corpus, sep='\t')

assoc_corpus = "/compLing/students/hiwi-theses/data/aylin.wahl/MUDCATData/SWOW-EN.R100.csv"
data = {"ASSOC":['CS']}

#itterate through assocs
with open(assoc_corpus, "r") as asslist:
    for assline in asslist:
        assline = re.sub(r'"', "", assline)  #access SWOWtargets
        ass = assline.split(",")[9]       
        ass = re.sub(r'\n', "", ass) 
#itterate through targets and check if ass is in targets
        if ass in targetdata['TARGET'].values: 
            assind = targetdata[targetdata['TARGET']== ass].index.item() #accessing target index
            cs = targetdata['CS'].values[assind]  #accessing target CS
            data[ass] = [cs]                      #adding cs to target
            ass1 = assline.split(",")[10]  #access SWOWassocs + cleaning
            ass1 = re.sub(r'\n', "", ass1) 
            ass2 = assline.split(",")[11]
            ass2 = re.sub(r'\n', "", ass2) 
            ass3 = assline.split(",")[12]
            ass3 = re.sub(r'\n', "", ass3) 
            if ass1 not in data['ASSOC']: #assign SWOWassocs to colunns 
                data["ASSOC"].append(ass1)
            if ass2 not in data['ASSOC']:
                data["ASSOC"].append(ass2)
            if ass3 not in data['ASSOC']:
                data["ASSOC"].append(ass3)
df = pd.DataFrame.from_dict(data, orient ='index')
df = df.replace([None], 0)
print('DONE, Next Step :)')

In [ ]:
#  [3] Tokenassociation Tables
#filling existent pandas(all values zero) with counts
ASSOClist = list(data['ASSOC']) #get list of associations

with open(assoc_corpus, "r") as assoclist:
    for assocline in assoclist:
            assocline = re.sub(r'"', "", assocline)  #access assoctarget
            ass = assocline.split(",")[9]       
            ass = re.sub(r'\n', "", ass)
            if ass in data:
                ass1 = assocline.split(",")[10]  #access R1assocs + cleaning
                ass1 = re.sub(r'\n', "", ass1)
                assind = ASSOClist.index(ass1)          #getting column index
                df.loc[ass,assind] += 1                 #count target-assoc pair
                ass2 = assocline.split(",")[11]  #access R2assocs + cleaning
                ass2 = re.sub(r'\n', "", ass2)
                assind = ASSOClist.index(ass2)          
                df.loc[ass,assind] += 1
                ass3 = assocline.split(",")[12]  #access R3assocs + cleaning
                ass3 = re.sub(r'\n', "", ass3)
                assind = ASSOClist.index(ass3)          
                df.loc[ass,assind] += 1
#assocs as header 
df.columns = df.iloc[0]
df = df[1:]         
df.to_csv('/compLing/students/hiwi-theses/data/aylin.wahl/MUDCATData/original_data/targets_Brysbaert_filtered/adj_filtered_AssocTable.csv') 


# TokenAssoxiationTablesCHECKING OUTPUT:

-targets get 100 counts each for R1 -> check

In [ ]:
#print(df)
df.columns = df.iloc[0]
df = df[1:]
df.reset_index(drop=True, inplace=True)
print(df)

In [ ]:
#get target matches with SWOW 

import pandas as pd
 
table = '/compLing/students/hiwi-theses/data/aylin.wahl/MUDCATData/AssocTable.csv'

table = pd.read_csv(table)

index_list = list(table.index.values)
print(index_list)

#getting assoc. amounts fo each target
print(df.sum(axis = 1, skipna = True))


# Type Asscoiation Tables


In [ ]:
# Built Type AsscoiationTable with all zeros
# columns= from lemmatization of the tokens in the columns of the original tokenassociationtable
# rows = targets same as in the original tokensassociationtable
import pandas as pd
import nltk
import numpy as np
import re

#datatype= 'extreme'
datatype= 'filtered'
#wordclass = 'adj'
wordclass = 'nouns'
#wordclass = 'verbs'

#original tokenassociationtables
assoctable = pd.read_csv('/compLing/students/hiwi-theses/data/aylin.wahl/MUDCATData/original_data/targets_Brysbaert_'+datatype+'/R1only_'+wordclass+'_'+datatype+'_AssocTable.csv')

lemma = nltk.WordNetLemmatizer()

tokensassoctable = list(assoctable.columns)
lemassoctable = list(map(lemma.lemmatize, tokensassoctable)) # lemmatizing tokenfeatures so only types are left
print(len(set(lemassoctable))-2) #number of types, unifying the types so no type is mentioned twice in list

targets = list(assoctable['Unnamed: 0'])
lentargets = len(targets)
CS = list(assoctable['CS'])  

print(len(list(assoctable['Unnamed: 0'])))


data = pd.DataFrame(index=np.arange(len(list(assoctable['Unnamed: 0']))), columns=set(lemassoctable))  #types in columns, index by target length
cols_to_order = ['Unnamed: 0', 'CS']
new_columns = cols_to_order + (data.columns.drop(cols_to_order).tolist())  
data = data[new_columns]   #setting ordered columns 
data['Unnamed: 0']=targets #assigning targets to column
data['CS']=CS              #assigning CS to column
data['']= np.zeros(lentargets) #adding NA column for empty R1/R2/R3
df = data.replace([None], 0)

print(df)


In [ ]:
# #  [3] Typeassociation Tables
#filling existent pandas(all values zero) with counts
assoc_corpus = "/compLing/students/hiwi-theses/data/aylin.wahl/MUDCATData/original_data/SWOW-EN.R100.csv"
ASSOClist = list(df.columns) #get list of associations
ASSOCtarget = list(df['Unnamed: 0'])  #get list of targets

with open(assoc_corpus, "r") as assoclist:
    for assocline in assoclist:
            assocline = re.sub(r'"', "", assocline)  #access assoctarget
            ass = assocline.split(",")[9]       
            ass = re.sub(r'\n', "", ass)
           # print('ASS: '+ass)
            if ass in targets:
                ass1 = assocline.split(",")[10]  #access R1assocs + cleaning
                ass1 = re.sub(r'\n', "", ass1)
               # print('ASS1: ' + str(ass1))
                #print('ASS: '+str(ass))
                ass1 = lemma.lemmatize(ass1)
                assind = ASSOClist.index(ass1)          #getting column index
                assindt = ASSOCtarget.index(ass)        #getting row index
               # print('ASS1: ' + str(ass1))
               # print('ASS: '+str(ass))
                df.iloc[assindt,assind] += 1                 #count target-assoc pair
               # ass2 = assocline.split(",")[11]  #access R2assocs + cleaning
               # ass2 = re.sub(r'\n', "", ass2)
               # ass2 = lemma.lemmatize(ass2)
               # assind = ASSOClist.index(ass2) 
                #print('ASSINDT: '+str(assindt))
                #print('ASS1: ' + str(ass1))
               # print('ASS2: ' + str(ass2))
              #  df.iloc[assindt,assind] += 1
              #  ass3 = assocline.split(",")[12]  #access R3assocs + cleaning
              #  ass3 = re.sub(r'\n', "", ass3)
              #  ass3 = lemma.lemmatize(ass3)
              #  assind = ASSOClist.index(ass3)          
              #  df.iloc[assindt,assind] += 1
                
#assocs as header 
#df.columns = df.iloc[0]
#df = df[1:]         
df.to_csv('/compLing/students/hiwi-theses/data/aylin.wahl/MUDCATData/original_data/targets_Brysbaert_'+datatype+'/R1only_types_'+wordclass+'_'+datatype+'_AssocTable.csv')



In [ ]:
df

# TypeAssoxiationTablesCHECKING OUTPUT:

-targets get 100 counts each for R1 -> check

In [ ]:
#checking if assoctbale correct

#getting assoc. amounts fo each target, 100 people were asked one target in SWOW and gave 3 answers so 300 count per target 
print(df.sum(axis = 1).nsmallest(n=10))
#print(df.nsmallest(n=10))

# Operations fixing individual stuff

In [ ]:
# fixing types assoctables (fixed with index_FALSE) doesnt have to be run again

import pandas as pd

datatype= 'extreme'
#datatype= 'filtered'
wordclass = 'adj'
#wordclass = 'nouns'
#wordclass = 'verbs'

#original tokenassociationtables
assoctable = pd.read_csv('/compLing/students/hiwi-theses/data/aylin.wahl/MUDCATData/original_data/targets_Brysbaert_'+datatype+'/types_'+wordclass+'_'+datatype+'_AssocTable.csv')
df = assoctable.drop(columns=['Unnamed: 0.1'])
df.to_csv('/compLing/students/hiwi-theses/data/aylin.wahl/MUDCATData/original_data/targets_Brysbaert_'+datatype+'/types_'+wordclass+'_'+datatype+'_AssocTable.csv',index=False)

In [ ]:
# changig one value in SWOW

import pandas as pd
assoctable = pd.read_csv("/compLing/students/hiwi-theses/data/aylin.wahl/MUDCATData/original_data/SWOW-EN.R100.csv")
#print(assoctable[133802:133803])
#assoctable.loc[133802, 'R1'] = 'pocket'
#print(assoctable[133802:133803])

#assoctable.to_csv("/compLing/students/hiwi-theses/data/aylin.wahl/MUDCATData/original_data/SWOW-EN.R100.csv")
assoc = assoc.replace(r'^\s*$', np.nan, regex=True)
assoc.to_csv("/compLing/students/hiwi-theses/data/aylin.wahl/MUDCATData/original_data/SWOW-EN.R100.csv", index=False)


# One-Hot AssociationTables

In [ ]:
# one-hot of token assoctables

import pandas as pd

#datatype= 'extreme'
datatype= 'filtered'
#wordclass = 'adj'
wordclass = 'nouns'
#wordclass = 'verbs'

#original tokenassociationtables
assoctable = pd.read_csv('/compLing/students/hiwi-theses/data/aylin.wahl/MUDCATData/original_data/targets_Brysbaert_'+datatype+'/'+wordclass+'_'+datatype+'_AssocTable.csv')
df = assoctable.drop(columns=['Unnamed: 0', 'CS'])
df = df<1   #booleans for the numbers 
df = df.replace({True: 0, False: 1})  #booleans in 0 and 1
df.insert(0, 'CS', assoctable['CS'])  #CS back in dataframe
df.insert(0, 'Unnamed: 0', assoctable['Unnamed: 0'])  #taregts back in dataframe

df.to_csv('/compLing/students/hiwi-theses/data/aylin.wahl/MUDCATData/original_data/targets_Brysbaert_'+datatype+'/one-hot_'+wordclass+'_'+datatype+'_AssocTable.csv',index=False)


In [ ]:
# one-hot of types assoctables

import pandas as pd

datatype= 'extreme'
#datatype= 'filtered'
wordclass = 'adj'
#wordclass = 'nouns'
#wordclass = 'verbs'

#original tokenassociationtables
assoctable = pd.read_csv('/compLing/students/hiwi-theses/data/aylin.wahl/MUDCATData/original_data/targets_Brysbaert_'+datatype+'/types_'+wordclass+'_'+datatype+'_AssocTable.csv')
print(assoctable)
df = assoctable.drop(columns=['Unnamed: 0', 'CS'])
df = df<1   #booleans for the numbers 
df = df.replace({True: 0, False: 1})  #booleans in 0 and 1
df.insert(0, 'CS', assoctable['CS'])  #CS back in dataframe
df.insert(0, 'Unnamed: 0', assoctable['Unnamed: 0'])  #taregts back in dataframe
print(df)

df.to_csv('/compLing/students/hiwi-theses/data/aylin.wahl/MUDCATData/original_data/targets_Brysbaert_'+datatype+'/one-hot_types_'+wordclass+'_'+datatype+'_AssocTable.csv',index=False)

# Type Assoctable without NA

In [2]:
# droping NA columns in the type assoctables

import pandas as pd

#datatype = 'extreme'
datatype = 'filtered'
wordclass = 'nouns'
#wordclass = 'verbs'
#wordclass = 'adj'

df = pd.read_csv('/compLing/students/hiwi-theses/data/aylin.wahl/MUDCATData/original_data/targets_Brysbaert_'+datatype+'/R1only_types_'+wordclass+'_'+datatype+'_AssocTable.csv')
print(len(list(df.columns)))
NAtable = pd.DataFrame(columns=['CS','NA Amount'])
NAtable['CS'] = df['CS']
NAtable['NA Amount'] = df['NA']
df = df.drop('NA', axis=1)  #dropping the NA column
df = df.drop(df.iloc[:,-1:],axis=1)  #dropping the last column of the table !!!!AFTER CHECKING DATATABLES LAST COLUMN
print(len(list(df.columns)))



df.to_csv('/compLing/students/hiwi-theses/data/aylin.wahl/MUDCATData/original_data/targets_Brysbaert_'+datatype+'/R1only_noNAtypes_'+wordclass+'_'+datatype+'_AssocTable.csv',index=False)
NAtable.to_csv('/compLing/students/hiwi-theses/data/aylin.wahl/MUDCATData/original_data/targets_Brysbaert_'+datatype+'/R1only_NAtypes_'+wordclass+'_'+datatype+'.csv',index=False)



29000


KeyError: ''

# L1 Normalizing without NA tables

In [ ]:
# for datatables where the sum of associations isnt equal for all targets (bc NAs deleted)
# dividing every cell with the amount of associatins for this target

import pandas as pd
from sklearn import preprocessing

#datatype= 'extreme'
datatype= 'filtered'
wordclass = 'adj'
#wordclass = 'nouns'
#wordclass = 'verbs'

df = pd.read_csv('/compLing/students/hiwi-theses/data/aylin.wahl/MUDCATData/original_data/targets_Brysbaert_'+datatype+'/noNA'+wordclass+'_'+datatype+'_AssocTable.csv')
cs = df['CS']
target = df['Unnamed: 0']
df = df.drop('CS', axis=1)  #dropping the CS column
df = df.drop('Unnamed: 0', axis=1)  #dropping the Target column

#data = preprocessing.normalize(df, axis=1) # normalize df rows
df = df.div(df.sum(axis=1), axis=0) # summing row and dividing row by that sum
df = pd.DataFrame(df, columns=df.columns)
df.insert(0, 'CS', cs) #adding CS again
df.insert(0, 'Unnamed: 0', target) #adding targets again

df.to_csv('/compLing/students/hiwi-theses/data/aylin.wahl/MUDCATData/original_data/targets_Brysbaert_'+datatype+'/NORMnoNA_'+wordclass+'_'+datatype+'_AssocTable.csv',index=False)             
  
# checking if sum of rows is CS+1.0
print(df.sum(axis=1))
print(cs)

# Word Class Vectors
- no NAs
- normalized

In [ ]:
# word classes as features, POS counts in each cell per target
import pandas as pd

#datatype= 'extreme'
datatype= 'filtered'
wordclass = 'adj'
#wordclass = 'nouns'
#wordclass = 'verbs'

df = pd.read_csv('/compLing/students/hiwi-theses/data/aylin.wahl/MUDCATData/M4_POStable/'+wordclass+'_'+datatype+'_POStable.csv')

# delete NA POS tags
df = df.drop('SYM', axis=1)  #dropping the SYM column
df = df.drop('.', axis=1)  #dropping the SYM column
df = df.drop(':', axis=1)  #dropping the SYM column
df = df.drop('$', axis=1)  #dropping the SYM column



In [ ]:
print(df.sum(axis=1))

# getting stats about the targets

In [ ]:
# amount of targets before assocaition-is-in-swow filter
import pandas as pd

target_adj = "/compLing/students/hiwi-theses/data/aylin.wahl/MUDCATData/original_data/targets_Brysbaert_filtered/targets_Brysbaert_filtered_adj.txt"
target_verb = "/compLing/students/hiwi-theses/data/aylin.wahl/MUDCATData/original_data/targets_Brysbaert_filtered/targets_Brysbaert_filtered_verbs.txt"
target_noun = "/compLing/students/hiwi-theses/data/aylin.wahl/MUDCATData/original_data/targets_Brysbaert_filtered/targets_Brysbaert_filtered_nouns.txt"

target_adj = pd.read_csv(target_adj, sep='\t')
print('ADJ: ' +str(len(target_adj.index)))
      
target_verb = pd.read_csv(target_verb, sep='\t')
print('VERB: ' +str(len(target_verb.index)))
      
target_noun = pd.read_csv(target_noun, sep='\t')
print('NOUN: ' +str(len(target_noun.index)))




In [ ]:
# amount of targets after association-is-in-swow filter

dfa = pd.read_csv('/compLing/students/hiwi-theses/data/aylin.wahl/MUDCATData/original_data/targets_Brysbaert_filtered/adj_filtered_AssocTable.csv')
dfv = pd.read_csv('/compLing/students/hiwi-theses/data/aylin.wahl/MUDCATData/original_data/targets_Brysbaert_filtered/verbs_filtered_AssocTable.csv')
dfn = pd.read_csv('/compLing/students/hiwi-theses/data/aylin.wahl/MUDCATData/original_data/targets_Brysbaert_filtered/nouns_filtered_AssocTable.csv')

dfaex = pd.read_csv('/compLing/students/hiwi-theses/data/aylin.wahl/MUDCATData/original_data/targets_Brysbaert_extreme/adj_extreme_AssocTable.csv')
dfvex = pd.read_csv('/compLing/students/hiwi-theses/data/aylin.wahl/MUDCATData/original_data/targets_Brysbaert_extreme/verbs_extreme_AssocTable.csv')
dfnex = pd.read_csv('/compLing/students/hiwi-theses/data/aylin.wahl/MUDCATData/original_data/targets_Brysbaert_extreme/nouns_extreme_AssocTable.csv')


print('ADJ: ' +str(len(dfa.index)))    
print('VERB: ' +str(len(dfv.index)))    
print('NOUN: ' +str(len(dfn.index)))

print('ADJ: ' +str(len(dfaex.index)))    
print('VERB: ' +str(len(dfvex.index)))    
print('NOUN: ' +str(len(dfnex.index)))


In [ ]:
# feature reduction after lemmatization

dfanew = pd.read_csv('/compLing/students/hiwi-theses/data/aylin.wahl/MUDCATData/original_data/targets_Brysbaert_filtered/noNAtypes_adj_filtered_AssocTable.csv')
dfvnew = pd.read_csv('/compLing/students/hiwi-theses/data/aylin.wahl/MUDCATData/original_data/targets_Brysbaert_filtered/noNAtypes_verbs_filtered_AssocTable.csv')
dfnnew = pd.read_csv('/compLing/students/hiwi-theses/data/aylin.wahl/MUDCATData/original_data/targets_Brysbaert_filtered/noNAtypes_nouns_filtered_AssocTable.csv')

dfa = pd.read_csv('/compLing/students/hiwi-theses/data/aylin.wahl/MUDCATData/original_data/targets_Brysbaert_filtered/noNAadj_filtered_AssocTable.csv')
dfv = pd.read_csv('/compLing/students/hiwi-theses/data/aylin.wahl/MUDCATData/original_data/targets_Brysbaert_filtered/noNAverbs_filtered_AssocTable.csv')
dfn = pd.read_csv('/compLing/students/hiwi-theses/data/aylin.wahl/MUDCATData/original_data/targets_Brysbaert_filtered/noNAnouns_filtered_AssocTable.csv')

print('ADJ reduced: ' + str(len(dfanew.columns)-2) + ' / ' + str(len(dfa.columns)-2))
print('VERB reduced: ' + str(len(dfvnew.columns)-2) + ' / ' + str(len(dfv.columns)-2))
print('NOUN reduced: ' + str(len(dfnnew.columns)-2) + ' / ' + str(len(dfn.columns)-2))



In [ ]:
# na reduction percentage 
from statistics import mean

NAAfil = pd.read_csv('/compLing/students/hiwi-theses/data/aylin.wahl/MUDCATData/original_data/targets_Brysbaert_filtered/NAadj_filtered.csv')
NANfil = pd.read_csv('/compLing/students/hiwi-theses/data/aylin.wahl/MUDCATData/original_data/targets_Brysbaert_filtered/NAnouns_filtered.csv')
NAVfil = pd.read_csv('/compLing/students/hiwi-theses/data/aylin.wahl/MUDCATData/original_data/targets_Brysbaert_filtered/NAverbs_filtered.csv')

NAAfil = NAAfil['NA Amount'] /300
NANfil = NANfil['NA Amount'] /300
NAVfil = NAVfil['NA Amount'] /300

print('Adj: ' + str(mean(NAAfil)))
print('Verb: ' + str(mean(NAVfil)))
print('Noun: ' + str(mean(NANfil)))